(С) Андрей Кулинич, 2019

В целях кластеризации или обучения отдельных алгоритмов необходимо перевести категорийные признаки (оценки "хорошо", "плохо" и т.д. или названия ассортиментных групп) в количественные. В этом блокноте мы рассмотрим ряд подходов, которые вы сможете применить.

Импортируем библиотеку pandas.

In [15]:
import pandas as pd

Создадим DataFrame, который будет содержать как категорийные, так и количественные признаки.

In [12]:
df = pd.DataFrame({'Имя':['Иван', 'Петр', 'Мария', 'Ирина'],
                    'Пол':['М','М', 'Ж', 'Ж'], 
                    'Волосы':['Шатен', 'Блонд', 'Рыжий', 'Блонд'],
                    'Английский':['Хорошо', 'Хорошо', 'Отлично', 'Не владеет'],
                    'Возраст':[18, 19, 18, 20]})
df

,Имя,Пол,Волосы,Английский,Возраст
0,Иван,М,Шатен,Хорошо,18
1,Петр,М,Блонд,Хорошо,19
2,Мария,Ж,Рыжий,Отлично,18
3,Ирина,Ж,Блонд,Не владеет,20


## Самый простой способ

Самый простой способ предполагает замену всех категорийных признаков на фиктивную (dummy) переменную. Фиктивная переменная может принимать значение только 0 или 1. Смотрите на примере ниже. 

Мы создаем новый DataFrame df_with_dummies, где автоматически все категорийные переменные будут заменены фиктивными. Столбцы с категорийными переменными также автоматически будут удалены. Дальше уже можно работать с преобразованным датафреймом.

In [16]:
df_with_dummies = pd.get_dummies(df, columns=['Пол', 'Волосы', 'Английский'])
df_with_dummies

,Имя,Возраст,Пол_Ж,Пол_М,Волосы_Блонд,Волосы_Рыжий,Волосы_Шатен,Английский_Не владеет,Английский_Отлично,Английский_Хорошо
0,Иван,18,0,1,0,0,1,0,0,1
1,Петр,19,0,1,1,0,0,0,0,1
2,Мария,18,1,0,0,1,0,0,1,0
3,Ирина,20,1,0,1,0,0,1,0,0


В принципе, этого достаточно, но всегда можно лучше. Например, избежать лишних столбцов или получить другие эффекты.

## Чуть сложнее

Сформируем DataFrame df_work, где немного по-другому заменим столбцы с категорийными переменными. 

Во-первых, "Пол" имеет только два значения. А значит, закодировать его можем только одним столбцом. Для этого удалим одно из значений. Это можно сделать следующей командой. 

In [14]:
pd.get_dummies(df['Пол'], drop_first=True)

,М
0,1
1,1
2,0
3,0


Создадим столбец "Пол_код", где мужской пол будет закодирован единицей.

In [21]:
df_work=df #сделаем копию нашего исходного DataFrame
df_work['Пол_код']=pd.get_dummies(df_work['Пол'], drop_first=True)
df_work

,Имя,Пол,Волосы,Английский,Возраст,Пол_код
0,Иван,М,Шатен,Хорошо,18,1
1,Петр,М,Блонд,Хорошо,19,1
2,Мария,Ж,Рыжий,Отлично,18,0
3,Ирина,Ж,Блонд,Не владеет,20,0


Если цвет мы должны кодировать именно фиктивными переменными, то знание языка мы можем закодировать порядковыми номерами. Поясню: если цвет волос "Шатен" закодирован номером 0, "Блонд" - 2, "Рыжий" - 3, то это ошибка, так как это разные цвета. Между "Блонд" и "Шатен" расстояние не больше, чем между "Блонд" и "Рыжий". А вот между знанием хорошо и отлично английского языка действительно может быть меньше, чем полное незнание языка и отличное им владение. 

Можно использовать метод factorize: он автоматически сформирует список, который можно использовать как значения столбца.

In [22]:
pd.factorize(df['Английский'])

(array([0, 0, 1, 2], dtype=int64),
 Index(['Хорошо', 'Отлично', 'Не владеет'], dtype='object'))

Названия групп товаров можно кодировать через среднюю цену этой группы. Но можно и самим сформировать словарь с кодами. В этом случае вы управляете тем, что и под каким номером будет закодировано.

In [11]:
dic={'Не владеет':0, 'Плохо':1, 'Хорошо':2, 'Отлично':3}
df['Английский'].map(dic)

0    2
1    2
2    3
3    0
Name: Английский, dtype: int64

Итак, сформируем итоговый датасет df_work. 

Добавим столбец "Уровень английского".

In [23]:
df_work['Уровень английского']=df['Английский'].map(dic)
df_work

,Имя,Пол,Волосы,Английский,Возраст,Пол_код,Уровень английского
0,Иван,М,Шатен,Хорошо,18,1,2
1,Петр,М,Блонд,Хорошо,19,1,2
2,Мария,Ж,Рыжий,Отлично,18,0,3
3,Ирина,Ж,Блонд,Не владеет,20,0,0


Удалим столбцы "Английский" и "Пол".

In [25]:
df_work.drop(['Пол', 'Английский'], axis=1, inplace=True)
df_work

,Имя,Волосы,Возраст,Пол_код,Уровень английского
0,Иван,Шатен,18,1,2
1,Петр,Блонд,19,1,2
2,Мария,Рыжий,18,0,3
3,Ирина,Блонд,20,0,0


Закодируем dummy переменными цвет волос.

In [27]:
df_work=pd.get_dummies(df_work, columns=['Волосы'])
df_work

,Имя,Возраст,Пол_код,Уровень английского,Волосы_Блонд,Волосы_Рыжий,Волосы_Шатен
0,Иван,18,1,2,0,0,1
1,Петр,19,1,2,1,0,0
2,Мария,18,0,3,0,1,0
3,Ирина,20,0,0,1,0,0


Готово!